# Classification of Animals in 10 classes

# 1. Importing Libraries

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from keras.optimizers import Adam
from keras.losses import SparseCategoricalCrossentropy
import keras.callbacks as callbacks
from keras.preprocessing.image import ImageDataGenerator

import os
import random
from tqdm import tqdm
import matplotlib.pyplot as plt

## 2. Loading the dataset

Getting current working directory and dataset location

In [ ]:
current_path = os.getcwd()
img_path = os.path.join(current_path, 'raw-img')

Importing the dataset and defining train and test sets

In [ ]:
train = tf.keras.utils.image_dataset_from_directory(
    img_path,
    image_size=(224,224),
    batch_size=32,
    validation_split=0.2,
    subset='training',
    seed=21
    )

val = tf.keras.utils.image_dataset_from_directory(
    img_path,
    image_size=(224,224),
    batch_size=32,
    validation_split=0.2,
    subset='validation',
    seed=21
    )

Sampling image in order to check if the data is loaded correctly

Note that the labels are in italian and are traslated in english in the following code: <br>
"cane" = "dog", "cavallo" = "horse", "elefante" = "elephant", "farfalla" = "butterfly", "gallina" = "hen", \
"gatto" = "cat", "mucca" = "cow", "pecora" = "sheep", "ragno" = "spider", "scoiattolo" = "squirrel"

In [ ]:
img_num = random.randint(0,32)
sample = train.take(1).map(lambda x, y: (x[0],y[0])).as_numpy_iterator()
for i,j in sample:
    plt.imshow(i)
    # plt.title()
    plt.show()

## 3. Data Preprocessing

Normalizing the data for performance and better results

In [ ]:
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

train = train.map(process)
val = val.map(process)

## 4. Data Augmentation

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=20,
    horizontal_flip=True,
    brightness_range=[0.5, 1.5],
    zoom_range=[0.8, 1.2],
)

## 5. Building the model

In [ ]:
# Callbacks
# 1. ModelCheckpoint
# 2. EarlyStopping
# 3. ReduceLROnPlateau
# 4. TensorBoard

callbacks_list = [
    callbacks.ModelCheckpoint(
        filepath=os.path.join(current_path, 'models/kuz_model.{epoch:02d}-{val_loss:.2f}.h5'),
        monitor='val_loss',
        save_best_only=True
    ),

    callbacks.EarlyStopping(
        monitor='val_loss',
        min_delta=0.0001,
        patience=20,
        restore_best_weights=True,
    ),

    callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=20
    )
]

In [ ]:
callbacks_list2 = [
    callbacks.ModelCheckpoint(
        filepath=os.path.join(current_path, 'models/kuz2_model.{epoch:02d}-{val_loss:.2f}.h5'),
        monitor='val_loss',
        save_best_only=True
    ),
        
    callbacks.EarlyStopping(
        monitor='val_loss',
        min_delta=0.0001,
        patience=5,
        restore_best_weights=True,
    ),

    callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5
    )
]

In [ ]:
kuz_model = Sequential(
    [
        Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)),
        MaxPooling2D((2, 2), strides=(2, 2)),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2), strides=(2, 2)),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2), strides=(2, 2)),
        Flatten(),
        Dense(1024, activation='relu'),
        Dropout(0.2),
        Dense(10, activation='softmax')
    ]
)

kuz_model.summary()

kuz_model.compile(
    optimizer=Adam(learning_rate=0.0001),
    # loss=categorical_crossentropy,
    loss=SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)


In [ ]:
kuz_model2 = Sequential(
    [
        Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)),
        Conv2D(32, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2), strides=(2, 2)),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2), strides=(2, 2)),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2), strides=(2, 2)),
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        Flatten(),
        Dense(2048, activation='relu'),
        Dropout(0.2),
        Dense(10, activation='softmax')
    ]
)

kuz_model2.summary()

kuz_model2.compile(
    optimizer=Adam(learning_rate=0.0001),
    # loss=categorical_crossentropy,
    loss=SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)


In [ ]:
vgg19_model = Sequential(
    [
        Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        
        MaxPooling2D((2, 2), strides=(2, 2)),
        
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        
        MaxPooling2D((2, 2), strides=(2, 2)),
        
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        
        MaxPooling2D((2, 2), strides=(2, 2)),
        
        Conv2D(512, (3, 3), activation='relu', padding='same'),
        Conv2D(512, (3, 3), activation='relu', padding='same'),
        Conv2D(512, (3, 3), activation='relu', padding='same'),
        Conv2D(512, (3, 3), activation='relu', padding='same'),
        
        MaxPooling2D((2, 2), strides=(2, 2)),
        
        Conv2D(512, (3, 3), activation='relu', padding='same'),
        Conv2D(512, (3, 3), activation='relu', padding='same'),
        Conv2D(512, (3, 3), activation='relu', padding='same'),
        Conv2D(512, (3, 3), activation='relu', padding='same'),
        
        MaxPooling2D((2, 2), strides=(2, 2)),
        
        Flatten(),
        
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(4096, activation='relu'),
        
        Dense(10, activation='softmax')
    ]
)

vgg19_model.summary()

## 6. Model Training

In [ ]:
history1 = kuz_model.fit(
    train,
    validation_data=val,
    epochs=10,
    callbacks=callbacks_list
)

In [ ]:
history2 = kuz_model2.fit(
    train,
    validation_data=val,
    epochs=100,
    callbacks=callbacks_list2
)

## 7. Visualizing the models performance over the epochs

In [ ]:
#plotting the training and validation accuracy
plt.plot(history1.history['accuracy'])
plt.plot(history1.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

#plotting the training and validation loss
plt.plot(history1.history['loss'])
plt.plot(history1.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
#plotting the training and validation accuracy
plt.plot(history2.history['accuracy'])
plt.plot(history2.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

#plotting the training and validation loss
plt.plot(history2.history['loss'])
plt.plot(history2.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

## 8. Model Evaluation

## 9. Model Prediction

## 10. Saving the model

## 11. Loading the model

# The end